In [6]:
#!pip install --quiet glvis
import numpy as np
import matplotlib.pyplot as plt
from  bfp import *
import mfem.ser as mfem
#from mfem.common.arg_parser import ArgParser
from glvis import glvis, GlvisData
import seaborn as sns
import pandas as pd
import os

In [7]:
def create_2D_mesh(nx, ny, x_start, x_end, y_start, y_end):
    """Creates a 2D mesh with the specified intervals and coordinate ranges.

    Args:
        nx (int): Number of intervals in the x-direction.
        ny (int): Number of intervals in the y-direction.
        x_start (float): Starting x-coordinate.
        x_end (float): Ending x-coordinate.
        y_start (float): Starting y-coordinate.
        y_end (float): Ending y-coordinate.

    Returns:
        mesh: The updated mesh with vertex coordinates set accordingly.
    """

    # Generate equally spaced coordinates for x and y
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(y_start, y_end, ny + 1)
    
    # Create the mesh with initial x and y values set to 0
    mesh = mfem.Mesh(nx, ny, "QUADRILATERAL", True, 1.0, 1.0)
    
    # Retrieve the vertex array from the mesh
    verts = mesh.GetVertexArray()
    
    # Check if the number of vertices is as expected: (nx+1) * (ny+1)
    expected_num = (nx + 1) * (ny + 1)
    num_verts = mesh.GetNV()

    if num_verts != expected_num:
        print("Warning: Unexpected number of vertices! ({} != {})".format(num_verts, expected_num))
    
    # Update the vertex coordinates; vertices are stored in row-major order
    k = 0
    for j in range(ny + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1

    # Define the target directory and file name
    target_dir = os.path.join(os.getcwd(), 'mesh', 'usr')
    file_name = f'{nx}x{ny}_2D.mesh'
    file_path = os.path.join(target_dir, file_name)
    
    # Check if the target directory exists; if not, create it
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Directory '{target_dir}' was created.")

    # Check if the file already exists
    if not os.path.exists(file_path):
        # If the file does not exist, write the mesh to the file
        mesh.Print(file_path)
        print(f"File '{file_path}' was successfully created.")
    else:
        print(f"File '{file_path}' already exists.")

    return mesh

class EDependentCoefficient(mfem.PyCoefficient):
    def __init__(self, xs_t_data, E_start, E_end):
        super(EDependentCoefficient, self).__init__()
        self.E_start = E_start
        self.E_end = E_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.E_bins = np.linspace(E_start, E_end, self.n_groups + 1)

    def EvalValue(self, x):
        if self.constant:
            return self.constant_value
        E = x[1]
        for i in range(self.n_groups - 1):
            if np.isclose(E, self.E_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if E <= self.E_bins[i] and E > self.E_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

class XDependentCoefficient(mfem.PyCoefficient):

    def __init__(self, xs_t_data, x_start, x_end):
        super(XDependentCoefficient, self).__init__()
        self.x_start = x_start
        self.x_end = x_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.x_bins = np.linspace(x_start, x_end, self.n_groups + 1)

    def EvalValue(self, x):
        if self.constant:
            return self.constant_value

        x_val = x[0]
        for i in range(self.n_groups - 1):
            if np.isclose(x_val, self.x_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if self.x_bins[i] <= x_val < self.x_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

class MyVectorCoefficient(mfem.VectorConstantCoefficient):
    def __init__(self, mu_coeff, S_coeff):
        super(MyVectorCoefficient, self).__init__(2)
        self.mu_coeff = mu_coeff 
        self.S_coeff = S_coeff

    def EvalValue(self, x):
        mu_val = self.mu_coeff.EvalValue(x) 
        S_val = self.S_coeff.EvalValue(x) 
        return [mu_val, S_val]
    
class InflowCoefficientSN(mfem.PyCoefficient):
    def __init__(self, inflow, mu):
        super(InflowCoefficientSN, self).__init__()
        self.inflow = inflow
        self.mu = mu
    def EvalValue(self, x):
        #x_val = x[0]
        if self.mu > 0:
            return self.inflow
        else:
            return 0
        
class ConstantVector(mfem.VectorPyCoefficient):
    def __init__(self, constant):
        super(ConstantVector, self).__init__(1)
        self.constant = constant 
    def EvalValue(self, x):
        return self.constant

In [8]:
def test_EDependentCoefficient():
    xs_t_data = [0,1,2,3,4]
    coeff = EDependentCoefficient(xs_t_data, 1, 0.01)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.0, 0.604]))
    print(coeff.EvalValue([0.0, 0.406]))
    print(coeff.EvalValue([0.0, 0.208]))
    print(coeff.EvalValue([0.0, 0.01]))

def test_EDependentCoefficient_Constant():
    coeff = EDependentCoefficient(40, 0, 1)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([1.0, 0.604]))
    print(coeff.EvalValue([0.3, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

#print("Test for E dependent Coefficient")
#test_EDependentCoefficient()
#print("Test for E dependent Constant Coefficient")
#test_EDependentCoefficient_Constant()

def test_XDependentCoefficientt():
    xs_t_data = [0,1,2,3,4]
    coeff = XDependentCoefficient(xs_t_data, 0, 1)
    print(coeff.EvalValue([0.0, 0.00]))
    print(coeff.EvalValue([0.2, 0.00]))
    print(coeff.EvalValue([0.4, 0.00]))
    print(coeff.EvalValue([0.6, 0.00]))
    print(coeff.EvalValue([0.8, 0.00]))
    print(coeff.EvalValue([1.0, 0.00]))

def test_XDependentCoefficient_Constant():
    coeff = XDependentCoefficient(80, 0.0, 1.0)
    print(coeff.EvalValue([0.0, 0.00]))
    print(coeff.EvalValue([0.2, 0.00]))
    print(coeff.EvalValue([0.4, 0.00]))
    print(coeff.EvalValue([0.6, 0.00]))
    print(coeff.EvalValue([0.8, 0.00]))
    print(coeff.EvalValue([1.0, 0.00]))

#print("Test for X dependent Coefficient")
#test_XDependentCoefficientt()
#print("Test for X dependent Constant Coefficient")
#test_XDependentCoefficient_Constant()

def test_MyVectorCoefficient():
    mu_data = [5,6,7,8,9]
    mu = XDependentCoefficient(mu_data, 0.0, 1.0)
    S_data = [0,1,2,3,4]
    S = EDependentCoefficient(S_data, 1.0, 0.01)
    coeff = MyVectorCoefficient(mu, S)
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.2, 0.604]))
    print(coeff.EvalValue([0.4, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

#print("Test for Vector Coefficient")
#test_MyVectorCoefficient()


In [9]:
nx = 8
nE = 10
x_start = 0.0
x_end = 1.0
E_start = 0
E_end = 1.0
N_ang = 2
order = 1
inflow = 10

mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
dim = mesh.Dimension()

# btype = 0-GaussLegendre, 1 - GaussLobatto
fec = mfem.DG_FECollection(order, dim, btype=1)
fes = mfem.FiniteElementSpace(mesh, fec)
Size = fes.GetVSize()
print("Number of unknowns:", Size)

mu_vals, w_vals = gauss_legendre_dirs(N_ang)

#ex17'den
ess_tdof_list = mfem.intArray() 
dir_bdr = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr.Assign(0)
dir_bdr[2] = 1

S_const = 0.0
xs_t_const = 0.0
#xs_s_const = 1.0
S_arr = np.ones(nE) * S_const
E_arr = np.linspace(E_start, E_end, nE+1)

S_coeff = EDependentCoefficient(S_const, E_start, E_end)
xs_t_coeff = EDependentCoefficient(xs_t_const, E_start, E_end)
#xs_s_coeff = EDependentCoefficient(xs_s_const, E_start, E_end)
q_coeff = EDependentCoefficient(0.0, E_start, E_end)

# Solution

psi_mu = []
psi_mu_list = []

for mu, w in zip(mu_vals, w_vals):
    print("  Solving for mu =", mu)
    mu_coeff = ConstantCoefficient(mu)
    inflow_coeff = InflowCoefficientSN(inflow, mu)
    #marker = get_marker_for_mu(mesh, mu)
    v_coeff = MyVectorCoefficient(mu_coeff, S_coeff)
    print(f"inflow: {inflow_coeff.EvalValue([inflow, mu])}")
    print(f"v_coeff: {v_coeff.EvalValue([mu, 0.8])}")
    print(f"sigma_t: {xs_t_coeff.EvalValue([0.0, 0.604])}")

    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff, -1.0))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff, 1.0))
    b.Assemble()
    
    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)

    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()
    solver.SetOperator(A)
    solver.SetPreconditioner(prec)
    solver.SetRelTol(1e-12)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(1000)
    solver.SetKDim(30)
    solver.SetPrintLevel(1)
    solver.Mult(b, psi)
    

    g = glvis((mesh, psi), 500,500, keys='ARjmcbp')
    g.set_size(1000, 1000)
    g.render()
    
    #X_arr = mfem.GridFunction(fes)
    #X_arr.Assign(X)
    psi.Save("psi_hs_mu_{:.3f}.gf".format(mu))
    psi_mu.append(psi.GetDataArray())
    psi_mu_list.append((mu, w, psi))


phi = mfem.GridFunction(fes)
phi.Assign(0.0)
for mu, w, X in psi_mu_list:
    phi.Add(w, X)

phi.Save("phi_dg.gf")





    

File '/Users/melekderman/github/BFP/melek/mesh/mesh/usr/8x10_2D.mesh' already exists.
Number of unknowns: 320
  Solving for mu = -0.5773502691896257
inflow: 0
v_coeff: [-0.5773502691896257, 0.0]
sigma_t: 0.0
   Pass :  1   Iteration :   0  ||B r|| = 5570.75
   Pass :  1   Iteration :   1  ||B r|| = 332.002
   Pass :  1   Iteration :   2  ||B r|| = 78.0975
   Pass :  1   Iteration :   3  ||B r|| = 24.7499
   Pass :  1   Iteration :   4  ||B r|| = 13.432
   Pass :  1   Iteration :   5  ||B r|| = 10.308
   Pass :  1   Iteration :   6  ||B r|| = 8.57892
   Pass :  1   Iteration :   7  ||B r|| = 8.55541
   Pass :  1   Iteration :   8  ||B r|| = 8.1332
   Pass :  1   Iteration :   9  ||B r|| = 7.38502
   Pass :  1   Iteration :  10  ||B r|| = 1.98214
   Pass :  1   Iteration :  11  ||B r|| = 1.68485
   Pass :  1   Iteration :  12  ||B r|| = 0.885059
   Pass :  1   Iteration :  13  ||B r|| = 0.505738
   Pass :  1   Iteration :  14  ||B r|| = 0.504025
   Pass :  1   Iteration :  15  ||B r|| = 

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = 0.5773502691896257
inflow: 10
v_coeff: [0.5773502691896257, 0.0]
sigma_t: 0.0
   Pass :  1   Iteration :   0  ||B r|| = 50136.8
   Pass :  1   Iteration :   1  ||B r|| = 2988.02
   Pass :  1   Iteration :   2  ||B r|| = 702.877
   Pass :  1   Iteration :   3  ||B r|| = 222.749
   Pass :  1   Iteration :   4  ||B r|| = 120.888
   Pass :  1   Iteration :   5  ||B r|| = 92.772
   Pass :  1   Iteration :   6  ||B r|| = 77.2102
   Pass :  1   Iteration :   7  ||B r|| = 76.9987
   Pass :  1   Iteration :   8  ||B r|| = 73.1988
   Pass :  1   Iteration :   9  ||B r|| = 66.4652
   Pass :  1   Iteration :  10  ||B r|| = 17.8393
   Pass :  1   Iteration :  11  ||B r|| = 15.1637
   Pass :  1   Iteration :  12  ||B r|| = 7.96553
   Pass :  1   Iteration :  13  ||B r|| = 4.55271
   Pass :  1   Iteration :  14  ||B r|| = 4.53663
   Pass :  1   Iteration :  15  ||B r|| = 4.51802
   Pass :  1   Iteration :  16  ||B r|| = 3.44862
   Pass :  1   Iteration :  17  ||B r|| = 3.41211
   P

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [10]:
g = glvis((mesh, phi), 500,500, keys='ARjmcbp')
g.set_size(900, 900)
g.render()

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [11]:
g = glvis((mesh, psi), 500,500, keys='rljgac//0')
g.set_size(900, 900)
g.render()


_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…